## Network with multi-period battery dispatch


**Install PyPSA and PYPOWER**

In [17]:
pip install pypsa

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install pypower

Note: you may need to restart the kernel to use updated packages.


**Load IEEE 14-Bus Network**

In [20]:
import pypsa
import numpy as np
from pypower.api import case14

ppc = case14()
n = pypsa.Network()
n.import_from_pypower_ppc(ppc)

# Ensure we have at least one snapshot
if len(n.snapshots) == 0:
    n.set_snapshots([0])

# Run non-linear AC power flow
n.pf()

INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x7e757ebebec0> for snapshots Index(['now'], dtype='object', name='snapshot')


{'n_iter': name      0
 snapshot   
 now       4,
 'error': name                 0
 snapshot              
 now       6.235013e-13,
 'converged': name         0
 snapshot      
 now       True}

***Check results in PyPSA***

Bus voltage

In [21]:
# Voltage magnitudes in p.u. (index 0 = snapshot 0)
print("Bus voltage magnitudes (p.u.):")
print(n.buses_t.v_mag_pu.iloc[0])

print("\nBus voltage angles (rad):")
print(n.buses_t.v_ang.iloc[0])


Bus voltage magnitudes (p.u.):
name
1     1.060000
2     1.045000
3     1.010000
4     1.017671
5     1.019514
6     1.070000
7     1.061520
8     1.090000
9     1.055932
10    1.050985
11    1.056907
12    1.055189
13    1.050382
14    1.035530
Name: now, dtype: float64

Bus voltage angles (rad):
name
1     0.000000
2    -0.086963
3    -0.222095
4    -0.179994
5    -0.153133
6    -0.248202
7    -0.233169
8    -0.233169
9    -0.260726
10   -0.263497
11   -0.258145
12   -0.263119
13   -0.264527
14   -0.279840
Name: now, dtype: float64


Generator Output

In [22]:
print("\nGenerator active power (MW):")
print(n.generators_t.p.iloc[0])

print("\nGenerator reactive power (MVAr):")
print(n.generators_t.q.iloc[0])



Generator active power (MW):
name
G0    232.393272
G1     40.000000
G2      0.000000
G3      0.000000
G4      0.000000
Name: now, dtype: float64

Generator reactive power (MVAr):
name
G0   -16.549301
G1    43.557100
G2    25.075348
G3    12.730944
G4    17.623451
Name: now, dtype: float64


Line Flows

In [23]:
print("\nLine active power p0 (MW):")
print(n.lines_t.p0.iloc[0])

print("\nLine reactive power q0 (MVAr):")
print(n.lines_t.q0.iloc[0])



Line active power p0 (MW):
name
L0     1.568829e+02
L1     7.551038e+01
L2     7.323758e+01
L3     5.613150e+01
L4     4.151622e+01
L5    -2.328569e+01
L6    -6.115823e+01
L7     7.353277e+00
L8     7.786067e+00
L9     1.774798e+01
L10   -4.666581e-14
L11    2.807418e+01
L12    5.227552e+00
L13    9.426381e+00
L14   -3.785322e+00
L15    1.614258e+00
L16    5.643851e+00
Name: now, dtype: float64

Line reactive power q0 (MVAr):
name
L0    -20.404292
L1      3.854991
L2      3.560203
L3     -1.550350
L4      1.170998
L5      4.473116
L6     15.823642
L7      3.560473
L8      2.503414
L9      7.216575
L10   -17.162971
L11     5.778691
L12     4.219138
L13     3.610006
L14    -1.615063
L15     0.753959
L16     1.747174
Name: now, dtype: float64


***Output from PYPOWER - Double Check***

In [24]:
from pypower.api import case14, runpf, ppoption

ppc = case14()

# Create options: VERBOSE=0 (no text), OUT_ALL=0 (no final report)
ppopt = ppoption(VERBOSE=0, OUT_ALL=0)

ppc_res, success = runpf(ppc, ppopt)
print("PYPOWER success:", success)

bus_res = ppc_res["bus"]
gen_res = ppc_res["gen"]

# PYPOWER bus voltages:
v_mag_pp = bus_res[:, 7]   # Vm (p.u.)
v_ang_pp = bus_res[:, 8]   # Va (deg)

print("\nPYPOWER Vm (p.u.):")
print(v_mag_pp)
print("\nPYPOWER Va (deg):")
print(v_ang_pp)


PYPOWER success: 1

PYPOWER Vm (p.u.):
[1.06       1.045      1.01       1.01767085 1.01951386 1.07
 1.06151953 1.09       1.05593172 1.05098463 1.05690652 1.05518856
 1.05038171 1.03552995]

PYPOWER Va (deg):
[  0.          -4.98258914 -12.72509994 -10.31290109  -8.7738539
 -14.22094646 -13.35962737 -13.35962737 -14.9385213  -15.09728846
 -14.79062203 -15.07558452 -15.15627634 -16.03364453]


***Check Results***

In [25]:
print("PyPSA bus voltage magnitudes (p.u.):")
print(n.buses_t.v_mag_pu.iloc[0])

print("\nPyPSA bus voltage angles (rad):")
print(n.buses_t.v_ang.iloc[0])


PyPSA bus voltage magnitudes (p.u.):
name
1     1.060000
2     1.045000
3     1.010000
4     1.017671
5     1.019514
6     1.070000
7     1.061520
8     1.090000
9     1.055932
10    1.050985
11    1.056907
12    1.055189
13    1.050382
14    1.035530
Name: now, dtype: float64

PyPSA bus voltage angles (rad):
name
1     0.000000
2    -0.086963
3    -0.222095
4    -0.179994
5    -0.153133
6    -0.248202
7    -0.233169
8    -0.233169
9    -0.260726
10   -0.263497
11   -0.258145
12   -0.263119
13   -0.264527
14   -0.279840
Name: now, dtype: float64


***Compare PyPSA and PYPOWER***

In [26]:
# Make sure PYPOWER results are computed as in step 1
from pypower.api import runpf, ppoption

ppopt = ppoption(VERBOSE=0, OUT_ALL=0)
ppc_res, success = runpf(ppc, ppopt)

bus_res = ppc_res["bus"]
v_mag_pp = bus_res[:, 7]   # Vm p.u.
v_ang_pp = bus_res[:, 8]   # Va deg

# PyPSA results
v_mag_psa = n.buses_t.v_mag_pu.iloc[0].values
v_ang_psa = np.degrees(n.buses_t.v_ang.iloc[0].values)   # rad → deg

print("\nMax |ΔVm|:", np.max(np.abs(v_mag_psa - v_mag_pp)))
print("Max |ΔVa| (deg):", np.max(np.abs(v_ang_psa - v_ang_pp)))



Max |ΔVm|: 5.4791726711300726e-12
Max |ΔVa| (deg): 2.6198954117262474e-10
